# QED-CIS 
This notebook will can approximate the eigenstates of the Pauli-Fierz Hamiltonian in the coherent state basis using various configuration interaction approaches.  It utilizes a few different helper programs:

- `psi4` quantum chemistry engine
- `helper_cqed_rhf` performs RHF for the Pauli-Fierz Hamiltonian in the coherent state basis; provides orbital basis
- `helper_PFCI` contains a few classes, including `Determinant` and `PFHamiltonianGenerator` used to compute the CI matrix 



In [1]:
import time
import numpy as np
np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_cqed_rhf import cqed_rhf
# prepare the determinant list
from helper_PFCI import Determinant, PFHamiltonianGenerator, compute_excitation_level
from itertools import combinations



The PFHamiltonianGenerator will expect the different integral arrays to be in the spin orbital basis and will 
use physicist convention.  The functions below will be used to map arrays from spatial orbital basis and in chemist notation to spin orbital basis in physicist convention.  One in particular we need to attend to is the two-electron
dipole terms $d_{pq} d_{rs}$

In [2]:
def spin_idx_to_spat_idx_and_spin(P):
    """ function to take the numeric label of a spin orbital
        and return the spatial index and the spin index separately.
        Starts counting from 0:
        
        Arguments
        ---------
        P : int
            spin orbital label
        
        Returns
        -------
        [p, spin] : numpy array of ints
            p is the spatial orbital index and spin is the spin index.
            spin = 1  -> alpha
            spin = -1 -> beta
            
        Example
        -------
        >>> spin_idx_to_spat_idx_and_spin(0)
        >>> [0, 1]
        >>> spin_idx_to_spat_idx_and_spin(3)
        >>> [1, -1]
        
    """
    spin = 1
    if P % 2 == 0:
        p = P / 2
        spin = 1
    else:
        p = (P-1) / 2
        spin = -1
    return np.array([p, spin], dtype=int)


def map_spatial_to_spin(tei_spatial, I, J, K, L):
    """ function to take two electron integrals in the spatial orbital basis
        in chemist notation along with 4 indices I, J, K, L and return
        the corresponding two electron integral in the spin orbital basis
        in phycisit notation, <IJ||KL>
    
    """
    # Phys to Chem: <IJ||KL> -> [IK|JL] - [IL|JK]
    i_s = spin_idx_to_spat_idx_and_spin(I)
    k_s = spin_idx_to_spat_idx_and_spin(K)
    j_s = spin_idx_to_spat_idx_and_spin(J)
    l_s = spin_idx_to_spat_idx_and_spin(L)
    
    #print(i_s[1])
    # (ik|jl)
    spat_ikjl = tei_spatial[i_s[0], k_s[0], j_s[0], l_s[0]] * ( i_s[1] == k_s[1] ) *  ( j_s[1] == l_s[1] )
    
    # (il|jk)
    spat_iljk = tei_spatial[i_s[0], l_s[0], j_s[0], k_s[0]] * ( i_s[1] == l_s[1] ) *  ( j_s[1] == k_s[1] )
    
    return spat_ikjl - spat_iljk

def map_spatial_dipole_to_spin(mu, I, J, K, L):
    """ function to take the dipole matrix (a 1-electron matrix) 
        and return the product of dipole matrix elements such that it matches
        the <IJ||KL> convention.  
    
    """
    # Phys to Chem: <IJ||KL> -> [IK|JL] - [IL|JK]
    i_s = spin_idx_to_spat_idx_and_spin(I)
    k_s = spin_idx_to_spat_idx_and_spin(K)
    j_s = spin_idx_to_spat_idx_and_spin(J)
    l_s = spin_idx_to_spat_idx_and_spin(L)
    
    #print(i_s[1])
    # (ik|jl)
    spat_ikjl = mu[i_s[0], k_s[0]] * mu[j_s[0], l_s[0]] * ( i_s[1] == k_s[1] ) *  ( j_s[1] == l_s[1] )
    
    # (il|jk)
    spat_iljk = mu[i_s[0], l_s[0]] * mu[j_s[0], k_s[0]] * ( i_s[1] == l_s[1] ) *  ( j_s[1] == k_s[1] )
    
    return spat_ikjl - spat_iljk

In [3]:
# some test options
neglect_mumu = False
neglect_oe_terms = False

# Check energy against psi4?
compare_psi4 = True

# Run psi4 calculation and cqed-rhf calculation
This will give us a wavefunction object and a cqed-rhf orbital basis and associated
quantities for building the CQED-CI Hamiltonian later.

In [4]:


# Memory for Psi4 in GB
# psi4.core.set_memory(int(2e9), False)
psi4.core.set_output_file('output.dat', False)

# Memory for numpy in GB
numpy_memory = 2

mol_str = """
H 
H 1 1.0
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8
                  }



# photon energy
omega_val = 4.75 / psi4.constants.Hartree_energy_in_eV

# define the lambda vector
lambda_vector = np.array([0.1, 0.1, 0.1])

mol = psi4.geometry(mol_str)


psi4.set_options(options_dict)

print('\nStarting SCF and integral build...')
t = time.time()

# First compute SCF energy using Psi4
scf_e, wfn = psi4.energy('SCF', return_wfn=True)

# now compute cqed-rhf to get transformation vectors with cavity
cqed_rhf_dict = cqed_rhf(lambda_vector, mol_str, options_dict)

p4_wfn = cqed_rhf_dict["PSI4 WFN"]
ndocc = p4_wfn.doccpi()[0]
print(p4_wfn)



Starting SCF and integral build...

Start SCF iterations:

Canonical RHF One-electron energy = -2.2216883593010506
CQED-RHF One-electron energy      = -2.1941536730190530
Nuclear repulsion energy          = 0.5291772106700000
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -1.0385739629033175   dE = -1.03857E+00   dRMS = 1.01113E-16
SCF Iteration   2: Energy = -1.0385739629033175   dE =  0.00000E+00   dRMS = 1.16728E-16
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -1.03857396 hartree
Psi4  SCF energy: -1.06610865 hartree


# Parse quantities from CQED-RHF calculation
Quantities such as the CQED-RHF energy, orbital transformation matrix, and density
matrix are returned as dictionary entries from the CQED-RHF calculation.  This block parses 
those quantities and updates attributes of the wavefunction object with them.

In [5]:
# grab necessary quantities from cqed_rhf_dict
scf_e = cqed_rhf_dict["RHF ENERGY"]
cqed_scf_e = cqed_rhf_dict["CQED-RHF ENERGY"]
wfn = cqed_rhf_dict["PSI4 WFN"]
C = cqed_rhf_dict["CQED-RHF C"]
D = cqed_rhf_dict["CQED-RHF DENSITY MATRIX"]
eps = cqed_rhf_dict["CQED-RHF EPS"]
dc = cqed_rhf_dict["DIPOLE ENERGY"]

# collect rhf wfn object as dictionary
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# update wfn_dict with orbitals from CQED-RHF
wfn_dict["matrix"]["Ca"] = C
wfn_dict["matrix"]["Cb"] = C
# update wfn object
wfn = psi4.core.Wavefunction.from_file(wfn_dict)

# Grab data from wavfunction class
Ca = wfn.Ca()
ndocc = wfn.doccpi()[0]
nmo = wfn.nmo()

# Compute size of Hamiltonian in GB
from scipy.special import comb
nDet = comb(nmo, ndocc)**2
H_Size = nDet**2 * 8e-9
print('\nSize of the Hamiltonian Matrix will be %4.2f GB.' % H_Size)
if H_Size > numpy_memory:
    #clean()
    raise Exception("Estimated memory utilization (%4.2f GB) exceeds numpy_memory \
                    limit of %4.2f GB." % (H_Size, numpy_memory))

# Integral generation from Psi4's MintsHelper
t = time.time()
mints = psi4.core.MintsHelper(wfn.basisset())



Size of the Hamiltonian Matrix will be 0.00 GB.


/home/namvu/anaconda3/envs/p4dev/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Get integral arrays
This will get the various 1- and 2-electron integrals needed to compute the PF-CI Hamiltonian matrix elements.
We will get most of these in the ao basis and transform them using the MO coefficients (C-vectors) obtained from our
CQED-RHF calculation.  The last block also updated the C-vector attribute of psi4's wavefunction object, so we can
use some of the built-in transformation methods as well (e.g. `mints.mo_spin_eri(Ca, Ca)`)

In [6]:
# dipole arrays in AO basis
mu_ao_x = np.asarray(mints.ao_dipole()[0])
mu_ao_y = np.asarray(mints.ao_dipole()[1])
mu_ao_z = np.asarray(mints.ao_dipole()[2])

# transform dipole array to HF-PF basis
mu_cmo_x = np.dot(C.T, mu_ao_x).dot(C)
mu_cmo_y = np.dot(C.T, mu_ao_y).dot(C)
mu_cmo_z = np.dot(C.T, mu_ao_z).dot(C)

# electronic dipole expectation value with CQED-RHF density
mu_exp_x = np.einsum("pq,pq->", 2 * mu_ao_x, D)
mu_exp_y = np.einsum("pq,pq->", 2 * mu_ao_y, D)
mu_exp_z = np.einsum("pq,pq->", 2 * mu_ao_z, D)

# get electronic dipole expectation value
mu_exp_el = np.array([mu_exp_x, mu_exp_y, mu_exp_z])

# \lambda \cdot < \mu > where < \mu > contains only electronic terms 
l_dot_mu_exp = np.dot(lambda_vector, mu_exp_el)

# \lambda \cdot \mu_{el} in ao basis
l_dot_mu_el = lambda_vector[0] * mu_ao_x
l_dot_mu_el += lambda_vector[1] * mu_ao_y
l_dot_mu_el += lambda_vector[2] * mu_ao_z

# \lambda \cdot \mu_{el} in PF basis basis
l_dot_mu_el_cmo = lambda_vector[0] * mu_cmo_x
l_dot_mu_el_cmo += lambda_vector[1] * mu_cmo_y
l_dot_mu_el_cmo += lambda_vector[2] * mu_cmo_z

# get \lambda * <\mu>_e \lambda * \hat{\mu} term
d_PF = -l_dot_mu_exp * l_dot_mu_el

# quadrupole arrays
Q_ao_xx = np.asarray(mints.ao_quadrupole()[0])
Q_ao_xy = np.asarray(mints.ao_quadrupole()[1])
Q_ao_xz = np.asarray(mints.ao_quadrupole()[2])
Q_ao_yy = np.asarray(mints.ao_quadrupole()[3])
Q_ao_yz = np.asarray(mints.ao_quadrupole()[4])
Q_ao_zz = np.asarray(mints.ao_quadrupole()[5])


# Pauli-Fierz 1-e quadrupole terms, Line 2 of Eq. (9) in [McTague:2021:ChemRxiv]
Q_PF = -0.5 * lambda_vector[0] * lambda_vector[0] * Q_ao_xx
Q_PF -= 0.5 * lambda_vector[1] * lambda_vector[1] * Q_ao_yy
Q_PF -= 0.5 * lambda_vector[2] * lambda_vector[2] * Q_ao_zz

# accounting for the fact that Q_ij = Q_ji
# by weighting Q_ij x 2 which cancels factor of 1/2
Q_PF -= lambda_vector[0] * lambda_vector[1] * Q_ao_xy
Q_PF -= lambda_vector[0] * lambda_vector[2] * Q_ao_xz
Q_PF -= lambda_vector[1] * lambda_vector[2] * Q_ao_yz



Here we will build both the matrix of 1e integrals ($\langle p|\hat{O}_1|q \rangle$, where 
$\hat{O}_1$ contains electronic kinetic, nuclear-electron attraction, scaled quadrupole, and scaled dipole) in the AO basis so that they are (nao x nao) arrays, and we will build the $\langle pq||rs\rangle$ integrals augmented
by 2-electron scaled dipole integrals in the spin orbital 
basis which are (nso x nso x nso x nso) = (2 * nao x 2 * nao x 2 * nao x 2 * nao).

In [7]:
# H will contain the 1e- integrals
H = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential()) + Q_PF + d_PF



print('\nTotal time taken for ERI integrals: %.3f seconds.\n' % (time.time() - t))

#Make spin-orbital MO which will contain the 2e- integrals
print('Starting AO -> spin-orbital MO transformation...')
t = time.time()
# ERIs in spin-orbital basis in physicist convention
MO = np.asarray(mints.mo_spin_eri(Ca, Ca))

nso = 2 * nmo
TDI_spin = np.zeros((nso, nso, nso, nso))
# get the dipole-dipole integrals in the spin-orbital basis with physicist convention
for i in range(nso):
    for j in range(nso):
        for k in range(nso):
            for l in range(nso):
                TDI_spin[i, j, k, l] = map_spatial_dipole_to_spin(l_dot_mu_el_cmo, i, j, k, l)
                
# add dipole-dipole integrals to ERIs
MO += TDI_spin
    
print("Printing 1-e H matrix")   
print(H)
print(F'The dimensions of the 1-e H matrix is {H.shape}')
print(F'The dimesions of the 2-e integral arrays in the spin orbital basis is {MO.shape}')


Total time taken for ERI integrals: 0.012 seconds.

Starting AO -> spin-orbital MO transformation...
Printing 1-e H matrix
[[-0.96529 -0.67647]
 [-0.67647 -0.96529]]
The dimensions of the 1-e H matrix is (2, 2)
The dimesions of the 2-e integral arrays in the spin orbital basis is (4, 4, 4, 4)


Now we are going to transform 1-e integrals into the CQED-RHF basis

In [8]:

# Update H, transform to MO basis and tile for alpha/beta spin
H = np.einsum('uj,vi,uv', Ca, Ca, H)


# prepare the g matrix by transforming the l_dot_mu_el matrix * sqrt(\omega / 2)
g_mat = np.einsum('uj,vi,uv', Ca, Ca, -np.sqrt(omega_val / 2) * l_dot_mu_el)

print(H)
print(g_mat)




[[-1.09708  0.     ]
 [ 0.      -0.57361]]
[[-0.       0.03216]
 [ 0.03216  0.     ]]


Now we are going to put the 1-e arrays into the spin-orbital basis, so that each in the resulting
arrays is of the form $H_{pq} = \langle \chi_p | \hat{O}_1 | \chi_q \rangle$.  This array will be 
organized into spin blocks as follows:

\begin{equation}
\begin{bmatrix}
\langle p | \hat{O}_1 |q\rangle  & \langle p | \hat{O}_1| \overline{q}\rangle \\
\langle \overline{p} | \hat{O}_1|q\rangle  & \langle \overline{p}| \hat{O}_1| \overline{q}\rangle
\end{bmatrix}
\end{equation}

In [9]:
H = np.repeat(H, 2, axis=0)
print("printing H after 1st repeat")
print(H)
H = np.repeat(H, 2, axis=1)
print("printing H after 2nd repeat")
print(H)
g_mat = np.repeat(g_mat, 2, axis=0)
g_mat = np.repeat(g_mat, 2, axis=1)
print("printing g_mat after 2nd repeate")
print(g_mat)


# Make H block diagonal
spin_ind = np.arange(H.shape[0], dtype=np.int) % 2
H *= (spin_ind.reshape(-1, 1) == spin_ind)
g_mat *= (spin_ind.reshape(-1, 1) == spin_ind)

print("after spin blocking")
print(H)
print(g_mat)

print('..finished transformation in %.3f seconds.\n' % (time.time() - t))

printing H after 1st repeat
[[-1.09708  0.     ]
 [-1.09708  0.     ]
 [ 0.      -0.57361]
 [ 0.      -0.57361]]
printing H after 2nd repeat
[[-1.09708 -1.09708  0.       0.     ]
 [-1.09708 -1.09708  0.       0.     ]
 [ 0.       0.      -0.57361 -0.57361]
 [ 0.       0.      -0.57361 -0.57361]]
printing g_mat after 2nd repeate
[[-0.      -0.       0.03216  0.03216]
 [-0.      -0.       0.03216  0.03216]
 [ 0.03216  0.03216  0.       0.     ]
 [ 0.03216  0.03216  0.       0.     ]]
after spin blocking
[[-1.09708 -0.       0.       0.     ]
 [-0.      -1.09708  0.       0.     ]
 [ 0.       0.      -0.57361 -0.     ]
 [ 0.       0.      -0.      -0.57361]]
[[-0.      -0.       0.03216  0.     ]
 [-0.      -0.       0.       0.03216]
 [ 0.03216  0.       0.       0.     ]
 [ 0.       0.03216  0.       0.     ]]
..finished transformation in 0.014 seconds.



<ipython-input-9-050354007e26>:14: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  spin_ind = np.arange(H.shape[0], dtype=np.int) % 2


Here we are going to generate the set of all singly-excited determinants $|\Phi_i^a\rangle$
that we will call `singlesDets` and the reference determinant $|\Phi_0\rangle$ 
that we will calle `refDet`.

In principle we can use this loop to build a list of determinants for any CI method, including CASCI.
Right now it is actually generating all excitations (FCI) and then only computing those for which the total excitation rank is <= 1, which is equivalent to CIS.  

In [10]:

print('Generating singly-excited determinants')
t = time.time()
singlesDets = []
for alpha in combinations(range(nmo), ndocc): #<== this generates all combinations of alpha electrons in alpha orbitals
    alpha_ex_level = compute_excitation_level(alpha, ndocc) #<== this computes the excitation rank
    for beta in combinations(range(nmo), ndocc): #<== this generates all combinations of beta electronsi in betal orbitals
        beta_ex_level = compute_excitation_level(beta, ndocc) #<== this computes the excitation rank
        if alpha_ex_level + beta_ex_level <= 1: #<== this only proceeds if excitation rank is S or 0
            print(F' adding alpha: {alpha} and beta: {beta}\n') 
            singlesDets.append(Determinant(alphaObtList=alpha, betaObtList=beta))

print('..finished generating singly-excited determinants in %.3f seconds.\n' % (time.time() - t))
print(F'..there are {len(singlesDets)} determinants \n')
for i in range(len(singlesDets)):
    print(singlesDets[i])
    
print('Generating reference determinant')

occList = [i for i in range(ndocc)]
refDet = Determinant(alphaObtList=occList, betaObtList=occList)
print(refDet)


Generating singly-excited determinants
 adding alpha: (0,) and beta: (0,)

 adding alpha: (0,) and beta: (1,)

 adding alpha: (1,) and beta: (0,)

..finished generating singly-excited determinants in 0.000 seconds.

..there are 3 determinants 

|[0][0]>
|[0][1]>
|[1][0]>
Generating reference determinant
|[0][0]>


In [11]:
Np = 1
PF_Hamiltonian_Matrix = PFHamiltonianGenerator(Np, mol_str, options_dict , lambda_vector, omega_val)


Start SCF iterations:

Canonical RHF One-electron energy = -2.2216883593010506
CQED-RHF One-electron energy      = -2.1941536730190530
Nuclear repulsion energy          = 0.5291772106700000
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -1.0385739629033175   dE = -1.03857E+00   dRMS = 1.01113E-16
SCF Iteration   2: Energy = -1.0385739629033175   dE =  0.00000E+00   dRMS = 1.16728E-16
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -1.03857396 hartree
Psi4  SCF energy: -1.06610865 hartree
RAN CQED-RHF!
-1.0661086491853156


In [12]:
PF_H = PF_Hamiltonian_Matrix.generatePFMatrix(singlesDets)

DiffIn1Phot -> s:0, i:0, si:0, t:1, j:0, tj:3
DiffIn1Phot -> s:0, i:0, si:0, t:1, j:1, tj:4
DiffIn1Phot -> s:0, i:0, si:0, t:1, j:2, tj:5
DiffIn1Phot -> s:0, i:1, si:1, t:1, j:0, tj:3
DiffIn1Phot -> s:0, i:1, si:1, t:1, j:1, tj:4
DiffIn1Phot -> s:0, i:1, si:1, t:1, j:2, tj:5
DiffIn1Phot -> s:0, i:2, si:2, t:1, j:0, tj:3
DiffIn1Phot -> s:0, i:2, si:2, t:1, j:1, tj:4
DiffIn1Phot -> s:0, i:2, si:2, t:1, j:2, tj:5
DiffIn1Phot -> s:1, i:0, si:3, t:0, j:0, tj:0
DiffIn1Phot -> s:1, i:0, si:3, t:0, j:1, tj:1
DiffIn1Phot -> s:1, i:0, si:3, t:0, j:2, tj:2
DiffIn1Phot -> s:1, i:1, si:4, t:0, j:0, tj:0
DiffIn1Phot -> s:1, i:1, si:4, t:0, j:1, tj:1
DiffIn1Phot -> s:1, i:1, si:4, t:0, j:2, tj:2
DiffIn1Phot -> s:1, i:2, si:5, t:0, j:0, tj:0
DiffIn1Phot -> s:1, i:2, si:5, t:0, j:1, tj:1
DiffIn1Phot -> s:1, i:2, si:5, t:0, j:2, tj:2


In [13]:
# DiffIn1el1Phot -> s:0, i:1, si:1, t:1, j:2, tj:5
# DiffIn1el1Phot -> s:0, i:2, si:2, t:1, j:1, tj:4                    
print(PF_H)

[[-1.56775 -0.      -0.      -0.       0.03216  0.03216]
 [-0.      -1.04898  0.20864  0.03216  0.       0.     ]
 [-0.       0.20864 -1.04898  0.03216  0.       0.     ]
 [-0.       0.03216  0.03216 -1.39319 -0.      -0.     ]
 [ 0.03216  0.       0.      -0.      -0.87442  0.20864]
 [ 0.03216  0.       0.      -0.       0.20864 -0.87442]]


In [14]:
e_fci, wavefunctions = np.linalg.eigh(PF_H)
E_0 = -1.5677511735733174
for i in range(len(e_fci)):
    ex_e = e_fci[i]-E_0
    print(F'{ex_e:.12e}')

-2.287196326906e-03
1.708432671266e-01
3.101337811026e-01
4.846930628867e-01
7.311273851026e-01
9.042578485561e-01
